In [1]:
from models.swapnet import SWAP
import torch
from utils import  get_data_azimuths
from dataset import Seismic
import numpy as np

def v_y(y, label_delta=10):
        i = np.arange(0, 360, label_delta)
        index = np.arange(0,int(360/label_delta)).astype(np.int32)
        max_i = i[np.argmax(y)]
        if abs(max_i - 180) > 100:
            i = np.arange(-180, 180, label_delta)
            index -= int(180/label_delta)
        return np.sum(y[index] * i) % 360

def predict(model, x):
    x = torch.from_numpy(x).unsqueeze(0)
    with torch.no_grad():
        model.eval()
        pred_y = model(x, is_softmax = True).squeeze(0).numpy()
        pred_v = v_y(pred_y, label_delta=config.label_delta)
    return pred_v

class config:
    label_sigma = 10
    label_delta = 10

# 加载测试数据
data, azimuths = get_data_azimuths('data', 'test')
testset = Seismic(data, azimuths, signal_start=50, signal_length=200, 
                  label_sigma=config.label_sigma, 
                  label_delta=config.label_delta)

# 加载模型
model = SWAP(c_in = 3, c_out = 360//config.label_delta, nf=32, adaptive_size=25, ks1 = [17, 11, 5], ks2=7)
model.load_state_dict(torch.load('models/pretrained/swapnet.pt'))

<All keys matched successfully>

In [10]:
for test_index in np.random.randint(len(testset), size=(10,)):
    x, _ = testset[test_index]
    azimuth = azimuths[test_index]
    pred_azimuth = predict(model, x)
    print(f"Predicted azimuth:{pred_azimuth:.2f}, True azimuth:{azimuth:.2f}")

Predicted azimuth:241.34, True azimuth:241.28
Predicted azimuth:212.72, True azimuth:213.94
Predicted azimuth:142.73, True azimuth:130.73
Predicted azimuth:144.10, True azimuth:142.48
Predicted azimuth:278.46, True azimuth:282.21
Predicted azimuth:315.20, True azimuth:311.95
Predicted azimuth:144.63, True azimuth:140.14
Predicted azimuth:250.66, True azimuth:250.32
Predicted azimuth:311.74, True azimuth:281.43
Predicted azimuth:331.13, True azimuth:331.79
